In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pickle
from itertools import product

In [47]:
df = pd.read_csv('CAR DETAILS FROM CAR DEKHO.csv')
df = df.set_index('name') 
df['owner'] = df['owner'].apply(lambda x: 0 if x == "First Owner" else 1).astype(int) 
df['fuel'] = df['fuel'].apply(lambda x: 'Petrol' if x == 'Petrol' else ('Diesel' if x == 'Diesel' else 'Other'))
df = pd.get_dummies(df, columns=['fuel', 'seller_type', 'transmission']) 
df['year'] = 2023 - df['year']
df = df.drop(columns=['transmission_Automatic', 'seller_type_Trustmark Dealer', 'fuel_Other'])
df.columns.values[0] = 'Age'
df['selling_price'] = np.log10(df['selling_price']) 
df['km_driven'] = np.log10(df['km_driven'])
sp = df.pop('selling_price')
df.insert(0, 'selling_price', sp)
bool_columns = ['fuel_Diesel', 'fuel_Petrol', 'seller_type_Dealer', 'seller_type_Individual', 'transmission_Manual']
df[bool_columns] = df[bool_columns].astype(int)
df

,selling_price,Age,km_driven,owner,fuel_Diesel,fuel_Petrol,seller_type_Dealer,seller_type_Individual,transmission_Manual
name,,,,,,,,,
Maruti 800 AC,4.778151,16,4.845098,0,0,1,0,1,1
Maruti Wagon R LXI Minor,5.130334,16,4.698970,0,0,1,0,1,1
Hyundai Verna 1.6 SX,5.778151,11,5.000000,0,1,0,0,1,1
Datsun RediGO T Option,5.397940,6,4.662758,0,0,1,0,1,1
Honda Amaze VX i-DTEC,5.653213,9,5.149219,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...
Hyundai i20 Magna 1.4 CRDi (Diesel),5.612783,9,4.903090,1,1,0,0,1,1
Hyundai i20 Magna 1.4 CRDi,5.612783,9,4.903090,1,1,0,0,1,1
Maruti 800 AC BSIII,5.041393,14,4.919078,1,0,1,0,1,1


In [48]:
for i in range(3):
    df.iloc[:,i] = (df.iloc[:,i]-df.iloc[:,i].mean())/df.iloc[:,i].std() #Normalize for NN
df

,selling_price,Age,km_driven,owner,fuel_Diesel,fuel_Petrol,seller_type_Dealer,seller_type_Individual,transmission_Manual
name,,,,,,,,,
Maruti 800 AC,-2.099556,1.444908,0.379226,0,0,1,0,1,1
Maruti Wagon R LXI Minor,-1.133318,1.444908,0.005543,0,0,1,0,1,1
Hyundai Verna 1.6 SX,0.644016,0.258765,0.775346,0,1,0,0,1,1
Datsun RediGO T Option,-0.399121,-0.927378,-0.087060,0,0,1,0,1,1
Honda Amaze VX i-DTEC,0.301238,-0.215692,1.156934,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...
Hyundai i20 Magna 1.4 CRDi (Diesel),0.190316,-0.215692,0.527525,1,1,0,0,1,1
Hyundai i20 Magna 1.4 CRDi,0.190316,-0.215692,0.527525,1,1,0,0,1,1
Maruti 800 AC BSIII,-1.377334,0.970451,0.568410,1,0,1,0,1,1


In [52]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:],df.iloc[:,0], test_size = .2,random_state = 0) #Split

In [53]:
X_train

,Age,km_driven,owner,fuel_Diesel,fuel_Petrol,seller_type_Dealer,seller_type_Individual,transmission_Manual
name,,,,,,,,
Chevrolet Enjoy 1.3 TCDi LS 8,-0.452921,1.396851,0,1,0,0,1,1
Toyota Innova 2.5 VX (Diesel) 8 Seater BS IV,0.258765,1.364658,0,1,0,0,1,1
Maruti Esteem Vxi - BSIII,1.682136,0.881197,0,0,1,0,1,1
Honda City i VTEC V,-0.452921,0.322611,0,0,1,0,0,1
Chevrolet Beat LT Option,-0.690149,-0.214855,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...
Audi A6 2.0 TDI Premium Plus,0.021536,0.179910,0,1,0,1,0,0
Chevrolet Beat Diesel LS,0.258765,0.881197,0,1,0,0,1,1
Datsun RediGO S,-0.690149,-0.906231,0,0,1,0,1,1


In [64]:
hl1 = [(i,) for i in range(1,8)]

In [65]:
hl2 = list(product(range(1,8),range(1,8)))

In [66]:
HL = hl1+[(i,j) for i,j in hl2 if i>j]
HL

[(1,),
 (2,),
 (3,),
 (4,),
 (5,),
 (6,),
 (7,),
 (2, 1),
 (3, 1),
 (3, 2),
 (4, 1),
 (4, 2),
 (4, 3),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (6, 5),
 (7, 1),
 (7, 2),
 (7, 3),
 (7, 4),
 (7, 5),
 (7, 6)]

In [67]:
NN = MLPRegressor()
params = {'hidden_layer_sizes': HL, #It seems like 2 layers works better
          'activation': ['relu','logistic'], #Relu tends to work best
          'alpha': [0.001, 0.01],
          'max_iter':[1000]}
grid = GridSearchCV(estimator=NN, param_grid=params, scoring='r2', n_jobs = -1)
grid.fit(X_train, y_train)
print("Optimal Hyper-parameters: ", grid.best_params_)
print("Optimal Score: ", grid.best_score_)

Optimal Hyper-parameters:  {'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (6, 4), 'max_iter': 1000}
Optimal Score:  0.6988728018558168


In [71]:
HL = list(product(range(1,9),range(1,9)))
HL = [(i,j) for i,j in HL if i>=j]
NN = MLPRegressor()
params = {'hidden_layer_sizes': HL, 
          'activation': ['relu'],
          'alpha': [0.001, 0.01],
          'max_iter':[1000]}
grid = GridSearchCV(estimator=NN, param_grid=params, scoring='r2', n_jobs = -1)
grid.fit(X_train, y_train)
print("Optimal Hyper-parameters: ", grid.best_params_)
print("Optimal Score: ", grid.best_score_)

Optimal Hyper-parameters:  {'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (8, 6), 'max_iter': 1000}
Optimal Score:  0.7002485345389837


In [72]:
NN = MLPRegressor(hidden_layer_sizes=grid.best_params_['hidden_layer_sizes'],
                  activation=grid.best_params_['activation'],
                  alpha=grid.best_params_['alpha'], max_iter = 1000)
NN.fit(X_train, y_train)
y_pred = NN.predict(X_test)

In [73]:
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R2 Score:", r2_score(y_test, y_pred))

Mean Squared Error: 0.3166935921032444
R2 Score: 0.6945106810537394


In [44]:
#pickle.dump(NN, open("models/Regression_NN.pkl", "wb"))